# LSTM 情感分析

In [1]:
from keras.layers import Dense, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections #用来统计词频
import numpy as np
import re

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 预处理
获取每条评论的最大长度, 以及所有评论中不重复词的个数 

建立词到ID的映射, 保存在 word2ID

每条数据的形式如下:

1	The Da Vinci Code book is just awesome.

In [17]:
LSTM?

In [2]:
r1 = '[’\n·＊!"：#$%&\'()◆●（）＠②*+,-../:;<=>?@，。?★、…【】《》？——“”‘’！[\$$^_`{|}~]+'
r2 = '[\s]*'

label = [] ; text = []
max_lenth = 0   # 每个句子的最长长度, 40

with open(r'D:\CS\dataset\NLP\英文情感分类数据集(二分类).txt', 'r+', encoding='utf-8') as f:
    for line in f:
        line = line.lower()
        line = re.sub(r1, '', line)
        line = re.split(r2, line)
        if (len(line)-1) > max_lenth:
            max_lenth = (len(line)-1)
            
        label.append(int(line[0]))
        text.append(line[1:])
        
from gensim import corpora

word2ID = corpora.Dictionary(text)

D:\study\Anaconda\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
D:\study\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
num_word = len(word2ID.token2id.keys())

In [4]:
# 把每个文本中的词用ID表示, 每个文本长度为40, 不足的补零
vec_text = []
for i in text:
    temp_list = []
    for word in i:
        if word in word2ID.token2id.keys():
            temp_list.append(word2ID.token2id[word])
        else:
            temp_list.append(num_word+1)     # 不存在于字典的词用总词数+1填充
            
    bu0 = max_lenth - len(temp_list)
    temp_list.extend([num_word+2] * bu0)
    
    vec_text.append(temp_list)

In [5]:
train_X, test_X, train_y, test_y = train_test_split(np.array(vec_text), label, test_size=0.2)

## Model

In [6]:
embedding_size = 128
hidden_layer_size = 64

model = Sequential()
model.add(Embedding(2248+3, embedding_size))   # 将每个batch中的文本的每个词转化为词向量
model.add(LSTM(hidden_layer_size, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

↑为什么第5行Embedding层要+3而不是+2?

In [7]:
batch_size = 120
num_epoch = 5

model.fit(train_X, train_y, batch_size=batch_size, epochs=num_epoch, validation_data=(test_X, test_y))

Train on 5668 samples, validate on 1418 samples
Epoch 1/5
5668/5668 [==============================] - 4s 770us/step - loss: 0.6858 - acc: 0.5623 - val_loss: 0.6886 - val_acc: 0.5494
Epoch 2/5
5668/5668 [==============================] - 3s 552us/step - loss: 0.4754 - acc: 0.7424 - val_loss: 0.1324 - val_acc: 0.9647
Epoch 3/5
5668/5668 [==============================] - 3s 547us/step - loss: 0.1092 - acc: 0.9739 - val_loss: 0.0723 - val_acc: 0.9838
Epoch 4/5
5668/5668 [==============================] - 3s 556us/step - loss: 0.0340 - acc: 0.9929 - val_loss: 0.0553 - val_acc: 0.9873
Epoch 5/5
5668/5668 [==============================] - 3s 554us/step - loss: 0.0164 - acc: 0.9970 - val_loss: 0.0574 - val_acc: 0.9887


## toy

In [8]:
def text2vec(input_text):
    input_text = input_text.lower()
    input_text = re.sub(r1, '', input_text)
    input_text = re.split(r2, input_text)
    text_vec = []
    
    for word in input_text:
        if word in word2ID.token2id.keys():
            text_vec.append(word2ID.token2id[word])
        else:
            text_vec.append(num_word+1)     # 不存在于字典的词用总词数+1填充
            
    bu0 = max_lenth - len(text_vec)
    text_vec.extend([num_word+2] * bu0)
    return text_vec

In [14]:
input_text = 'You are so beautiful.'
text_vec = np.array(text2vec(input_text))
text_vec = text_vec.reshape(-1, text_vec.shape[0])

D:\study\Anaconda\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [16]:
model.predict_classes(text_vec)

array([[1]])

'You are so beautiful.'被正确预测为类别1